In [8]:
from ortools.sat.python import cp_model

In [15]:
array_hombre_a_mujer=[[3,5,4,2,1,6],[3,2,5,6,4,1],[2,4,3,1,5,6],[5,6,4,2,3,1],[2,5,3,6,4,1],[1,3,4,5,6,2]]
array_mujer_a_hombre=[[2,4,5,3,6,1],[3,5,4,2,1,6],[1,3,6,2,4,5],[3,2,5,6,4,1],[6,4,2,1,3,5],[6,4,3,1,5,2]]
nombres_hombres=["Richard","James","John","Bill","Greg","Mario"]
nombres_mujeres=["Helen","Tracy","Linda","Sally","Wanda","Mary"]
n=6

In [16]:
model = cp_model.CpModel()

#creación de las variables y dominios de hombres a mujeres
grilla_hombres = []
for i in range(n):
  fila = []
  for j in range(n):
    fila += [model.NewIntVar(0,1,'H'+str(i)+'_M'+str(j))]
  grilla_hombres+=[fila]

#creación de las variables y dominios de mujeres a hombres
grilla_mujeres = []
for i in range(n):
  fila = []
  for j in range(n):
    fila += [model.NewIntVar(0,1,'M'+str(i)+'_H'+str(j))]
  grilla_mujeres+=[fila]

#se unen las 2 grillas
grilla_total=[grilla_hombres]+[grilla_mujeres]

#restricciones
# 1 elige, 0 no elige
for i in range(n):
  #la fila solo puede sumar 1, ya que solo una persona puede elegir a una persona
  model.Add(sum(grilla_total[0][i])==1)
  model.Add(sum(grilla_total[1][i])==1)
  for j in range(n):
    #solo pueden elegir a personas con minimo de 4 orden de preferencia
    if(array_hombre_a_mujer[i][j]>4):
      model.Add(grilla_total[0][i][j]==0)
    if(array_mujer_a_hombre[i][j]>4):
      model.Add(grilla_total[1][i][j]==0)

for j in range(n):
  #tambien la columna solo pueden ser elegido por una persona
  c1 = [grilla_total[0][i][j] for i in range(n)]
  c2 = [grilla_total[1][i][j] for i in range(n)]
  model.Add(sum(c1)==1)
  model.Add(sum(c2)==1)

#las sumas de los valores de las transpuestas entre las 2 tablas no pueden sumar 1
for i in range(n):
  for j in range(n):
    model.Add(grilla_total[0][i][j]+grilla_total[1][j][i]!=1)

#solucionando el modelo
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 10.0
status = solver.Solve(model)

In [17]:
print("Hombres a Mujeres")
for i in range(n):
  for j in range(n):
    print(solver.Value(grilla_total[0][i][j]), end="")
  print("")

Hombres a Mujeres
000100
000001
010000
000010
001000
100000


In [18]:
print("Mujeres a Hombres")
for i in range(n):
  for j in range(n):
    print(solver.Value(grilla_total[1][i][j]), end="")
  print("")

Mujeres a Hombres
000001
001000
000010
100000
000100
010000


In [21]:
for i in range(n):
  for j in range(n):
    if(solver.Value(grilla_total[0][i][j])==1):
      print("'"+str(nombres_hombres[i])+"' se casa con '"+str(nombres_mujeres[j])+"'")

'Richard' se casa con 'Sally'
'James' se casa con 'Mary'
'John' se casa con 'Tracy'
'Bill' se casa con 'Wanda'
'Greg' se casa con 'Linda'
'Mario' se casa con 'Helen'
